In [3]:
%matplotlib ipympl
def figure(name, nrows=1, ncols=1, *args, **kwargs):
    plt.close(name)
    return plt.subplots(nrows, ncols, num=name, *args, **kwargs)

import numpy as np
import pylab as plt
plt.style.use('default')

In [41]:
from ipywidgets import IntSlider, Text, HBox, Layout
import warnings
warnings.filterwarnings('ignore')

In [42]:
displs = np.linspace(-0.02, 0.02, 9)
nums = [2 * i + 1 for i in range(15)]
ints = [0.1, 0.3, 0.7, 1., 1.3, 1.7, 2., 2.3, 2.7, 3.]
ws = [1.5, 2.2, 3.1]

In [43]:
# load data
data = np.zeros((3, len(ints), len(nums), len(displs)))
um = 1.5
for i, fs in enumerate(ints):
    data[0, i] = np.load(f'data/1c_2v_{um:.1f}um_fs={fs:.1f}Vnm.npz')['peaks']

In [47]:
def get_slider():
    fs = IntSlider(
        description='fs: ',
        value=0, min=0, max=len(ints)-1,
        readout=False
    )
    fs_text = Text(value=f'{ints[0]:.1f}V/nm', description='', layout=Layout(width='80px'))
    fs_ind = fs.value
    lines = []
    fig, ax = figure(1, 1, 3, figsize=(12, 4))
    for u in range(3):
        for i, num in enumerate(nums[:11]):
            peaks = data[0, fs_ind]
            lines.append(ax[u].plot(displs, peaks[i] / peaks[i, len(displs)//2], lw=2, label=f'HH{nums[i]}')[0])
        ax[u].set_xlabel('atom. displacement')
        ax[u].set_ylim([0.2, 2.02])
        ax[u].set_title(f'{ws[u]:.1f}um')
        
    def redraw(change):
        fs_ind = fs.value  
        for u in range(3):
            for i, num in enumerate(nums[:11]):
                peaks = data[0, fs_ind]
                lines[u*11 + i].set_data(displs, peaks[i] / peaks[i, len(displs)//2])
        fs_text.value = f'{ints[fs_ind]:.1f}V/nm'
        fig.canvas.draw()
        fig.canvas.flush_events()
        
    fs.observe(redraw, names='value')
    return HBox([fs, fs_text])

get_slider()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …